In [ ]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians
using Printf
using SparseArrays


In [ ]:
function extractSlice(dic, slice)
    dicSlice = Dict()
    numepxr = length(dic["ne"])
    for (key, value) in dic
        if length(value) == numepxr
            dicSlice[key] = value[slice]
        else
            dicSlice[key] = value
        end
    end
    return dicSlice
end

In [ ]:
function hasZeroDiag(M::SparseMatrixCSC{Tv, Ti}) where {Tv, Ti}
    for i in 1:size(M,1)
        if M[i,i] == zero(Tv)
            return true
        end
    end
    return false
end

function hasZeroDiag(name::String)
    M = downloadSS(name)
    return hasZeroDiag(M)
end

In [ ]:
@load "suitesparse_all.jl.split02merge02.jld2"
dic1 = copy(dic);

In [ ]:

sizeSlice = findall(dic1["ne"] .> 1000)
dic1 = extractSlice(dic1, sizeSlice);

In [ ]:
include("../julia-files/downloadSS.jl")

In [ ]:
zeroDiagSlice = findall(hasZeroDiag, dic1["testName"]);
noZeroDiagSlice = setdiff(1:length(sizeSlice), zeroDiagSlice);

In [ ]:
dic1 = extractSlice(dic1, noZeroDiagSlice);

In [ ]:
for name in dic1["testName"]
    println("- ", name)
end

In [ ]:
function nanErr!(d)
    for (key, value) in d
        if occursin("err", key)
            for i in 1:length(value)
                if isnan(value[i])
                    d[key][i] = Inf
                end
            end
        end
    end
end

In [ ]:
nanErr!(dic1);

In [ ]:
using Printf

function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))
ffK(s) = (@sprintf "%.4g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)
ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)

In [ ]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffmicro(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffmicro(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printMedian(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = median(d[string(solver, "_tot")] ./ d["ne"] )
        err = median(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function print75(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = quantile(d[string(solver, "_tot")] ./ d["ne"], 0.75)
        err = quantile(d[string(solver, "_err")], 0.75)
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function printMax(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = maximum(d[string(solver, "_tot")] ./ d["ne"])
        err = maximum(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

In [ ]:
function printSolverCell(d, solver) 
    return string(printMedian(d, solver), "&", print75(d, solver), "&", printMax(d, solver))
end

function printOneLine(d, solvers)
    allCells = [printSolverCell(d, solver) for solver in solvers]
    println(string(join(allCells, "&"), "\\\\"))
end

In [ ]:
solvers=["ac", "ac-s2m2", "cmg2"]
printOneLine(dic1, solvers)

In [ ]:
solvers = ["hypre", "petsc_hypre", "icc2"]
printOneLine(dic1, solvers)